Installation des packages


In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install langdetect

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from langdetect import detect
from IPython.display import display, HTML
import json

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# # open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)

Parcourir les fiches de métadonnées des catalogues ODATIS


In [ ]:
link_list=[(1,30),(31,60),(61,90),(91,120),(121,150)]

link_EN=[]
title_EN=[]
abstract_EN=[]

link_FR=[]
title_FR=[]
abstract_FR=[]

final_dic_EN = {}
final_dic_FR = {}
i=1
for element in link_list:
  source='https://www.odatis-ocean.fr/donnees-et-services/acces-aux-donnees/catalogue-complet#/search?fast=index&_content_type=json&from={}&to={}&sortBy=title&sortOrder=reverse'.format(element[0],element[1])
  print (source)
  driver.get(source)
  sleep(5)
  soup = BeautifulSoup(driver.page_source, "html.parser")
  table_rows=soup.find_all("a",class_="ng-binding ng-isolate-scope")
  #print (table_rows)

  for row in table_rows:
      print ('article n°: ',i)
      #EXTRAIRE LINKS
      link = str(row).find("href")
      sublink=str(row)[link+6:].split()[0].replace('"',"")
      links="https://www.odatis-ocean.fr"+sublink
      print (links)

      driver.get(links)
      sleep(5)
      #EXTRAIRE TITRE
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      for title in (soup.find_all("div",class_='col-md-8 gn-title ng-binding')):
          title_=title.get_text().strip()
      #EXTRAIRET LES TEXTES
      for abstract in (soup.find_all("div",class_='gn-abstract')):        
          abstract_=abstract.get_text().strip()
      #Detection de la langue
      lang=detect(abstract_)
      if lang=='en':

        link_EN.append(links)
        title_EN.append(title_)
        abstract_EN.append(abstract_)
      else:
        link_FR.append(links)
        title_FR.append(title_)
        abstract_FR.append(abstract_)
      
      i=i+1
      


final_dic_EN['Link']=link_EN
final_dic_EN['Title']=title_EN
final_dic_EN['Abstract']=abstract_EN

final_dic_FR['Lien']=link_FR
final_dic_FR['Titre']=title_FR
final_dic_FR['Abstrait']=abstract_FR

with open("final_output_ODATIS_EN.json", 'w') as fich:
        json.dump(final_dic_EN, fich)

with open("final_output_ODATIS_FR.json", 'w') as fich:
        json.dump(final_dic_FR, fich)

In [ ]:
#créer des fichiers csv
def pretty_print(df):
    return display(HTML(df.to_html()))

df_EN = pd.DataFrame(final_dic_EN)
pretty_print(df_EN)

print ('///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////')
df_FR = pd.DataFrame(final_dic_FR)
pretty_print(df_FR)

df_EN.to_csv(r'scrapped_data_from_ODATIS_EN.csv',index=False)
df_FR.to_csv(r'scrapped_data_from_ODATIS_FR.csv',index=False)

In [ ]:
#creer des fichiers txt
def create_corpus_from_txt(list_of_text : list, outputfn="output.txt"):
  """
  Transform a corpus (based on a list of str) into BiotexFormat
  
  Parameters
  ----------
  list_of_text : list
      list of stre
  outputfn : str, optional
      output filename (the default is "output.txt")
  
  """

  corpus=""
  sep="\n##########END##########\n"
  corpus = sep.join(list_of_text)
  open(outputfn,'w').write((corpus+sep).strip())

#create_corpus_from_txt(data["text2"].to_list(),outputfn="output.txt")
create_corpus_from_txt(df_EN.to_list(),outputfn="outputfr.txt")
create_corpus_from_txt(df_FR.to_list(),outputfn="outputen.txt")